# Goal

<h3 style="color:blue">assess the quality of summaries written by students</h3>
<h3 style="color:indigo">evaluate how well a student represents the main idea and details of a source text, as well as the clarity, precision, and fluency of the language used in the summary</h3>
<h3 style="color:red">Freely & publicly available external data is <b>allowed</b>, including pre-trained models</h3>
<h3>This is Multi-Output problem</h3>

### Use Hugging Face Library
### Use Tensorflow

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import math
import subprocess
from tqdm import tqdm
import pickle

In [3]:
import tensorflow as tf

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score, median_absolute_error

In [5]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [6]:
import transformers
from transformers import AutoTokenizer, TFBertModel

In [7]:
prompts_train = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')
summaries_train = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
prompts_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')
summaries_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')

In [8]:
train = pd.merge(prompts_train, summaries_train, on='prompt_id')
test = pd.merge(prompts_test, summaries_test, on='prompt_id')

In [9]:
train.rename(columns = {'text' : 'summary'}, inplace=True)
test.rename(columns = {'text' : 'summary'}, inplace=True)

In [10]:
train.head(2)

,prompt_id,prompt_question,prompt_title,prompt_text,student_id,summary,content,wording
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00791789cc1f,1 element of an ideal tragedy is that it shoul...,-0.210614,-0.471415
1,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,0086ef22de8f,The three elements of an ideal tragedy are: H...,-0.970237,-0.417058


In [11]:
columns_needed = ["prompt_text", "summary"]
train_data = train[columns_needed]
test_data = test[columns_needed]

In [12]:
def preprocess_text(text):
    # Tokenize the text using spaCy
    doc = nlp(text)
    
    preprocessed_words = [token.lemma_.lower().strip() for token in doc if not token.is_stop and token.is_alpha]
    
    preprocessed_text = ' '.join(preprocessed_words)
    
    return preprocessed_text



In [13]:
tqdm.pandas()

In [14]:
train_data[['preprocessed_prompt_text', 'preprocessed_summary']] = train_data.progress_apply(lambda row: row.apply(preprocess_text), axis=1)

100%|██████████| 7165/7165 [17:25<00:00,  6.85it/s]


In [15]:
test_data[['preprocessed_prompt_text', 'preprocessed_summary']] = test_data.progress_apply(lambda row: row.apply(preprocess_text), axis=1)

100%|██████████| 4/4 [00:00<00:00, 81.76it/s]


In [20]:
from transformers import AutoTokenizer, TFAutoModel
model = TFAutoModel.from_pretrained('/kaggle/input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased')

Some layers from the model checkpoint at /kaggle/input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at /kaggle/input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


### Next time use prepare_tf_dataset which is used to directly tokenize and data colat and
### make dataset compatible with tensorflow
####       https://huggingface.co/docs/transformers/v4.31.0/en/main_classes/model#transformers.TFPreTrainedModel.prepare_tf_dataset

In [21]:

def vectorize_dataframe(dataframe, col):
    vectors = []
    for text in tqdm(dataframe[col].tolist()):
        text_tokens = tokenizer(text, max_length=512, return_tensors="tf", padding=True, truncation=True)
        
        output = model(**text_tokens)
        array = np.array(output.pooler_output)
        vectors.append( array.flatten() )
        
    return vectors
    

In [22]:
train_data['prompt_text_embedded'] = vectorize_dataframe(train_data, 'preprocessed_prompt_text')

100%|██████████| 7165/7165 [24:05<00:00,  4.96it/s]


In [23]:
train_data['summary_embedded'] = vectorize_dataframe(train_data, 'preprocessed_summary')

100%|██████████| 7165/7165 [23:40<00:00,  5.04it/s]


In [24]:
test_data['prompt_text_embedded'] = vectorize_dataframe(test_data, 'preprocessed_prompt_text')
test_data['summary_embedded'] = vectorize_dataframe(test_data, 'preprocessed_summary')

100%|██████████| 4/4 [00:00<00:00,  5.13it/s]


In [25]:
with open("/kaggle/working/bert_prompt_text_preprocessed_embeddings.pkl", "wb") as file:
    pickle.dump(train_data['prompt_text_embedded'], file)
    
with open("/kaggle/working/bert_summary_preprocessed_embeddings.pkl", "wb") as file:
    pickle.dump(train_data['summary_embedded'], file)
    
#with open("/kaggle/input/embeddings/BERT_summary_embeddings.pkl", "rb") as file:
 #   train_data['summary_embedded'] = pickle.load(file)

In [26]:
traning_set = train_data[['prompt_text_embedded', 'summary_embedded']]
testing_set = test_data[['prompt_text_embedded', 'summary_embedded']]

### Take average of embeddings  [Not required, just checking]

In [27]:
target1 = np.array(train['content'])
target1 = target1.astype('float32')

target2 = np.array(train['wording'])
target2 = target2.astype('float32')

#target = (target1, target2)

In [28]:
def convert_tensor_to_numpy(tensor):
        return np.array(tensor, dtype='float32')

traning_set = traning_set.applymap(convert_tensor_to_numpy)
testing_set = testing_set.applymap(convert_tensor_to_numpy)

In [29]:
def prepare_dataset(dataset):
    # Flatten the nested arrays in the DataFrame
    dataset['prompt_text_embedded'] = dataset['prompt_text_embedded'].apply(lambda x: x.flatten())
    dataset['summary_embedded'] = dataset['summary_embedded'].apply(lambda x: x.flatten())
    
    feature1 = np.array(dataset['prompt_text_embedded'].tolist())
    feature2 = np.array(dataset['summary_embedded'].tolist())
    
    features = np.concatenate((feature1, feature2), axis=1)
    
    return features

In [30]:
features = prepare_dataset(traning_set)

In [31]:
features_for_test = prepare_dataset(testing_set)

In [32]:
#X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [33]:
from tensorflow.keras.layers import Dense, Input, Flatten

In [34]:
# Define the model architecture
model_content = tf.keras.Sequential([
      tf.keras.layers.Dense(356, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(232, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(94, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(1, activation='linear')
])

model_wording = tf.keras.Sequential([
     tf.keras.layers.Dense(356, activation='relu'),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Dense(232, activation='relu'),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Dense(94, activation='relu'),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Dense(1, activation='linear')
])

In [35]:
model_content.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])
model_wording.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

In [36]:
# Train your model using model.fit()
history1 = model_content.fit(features, target1, epochs=200, batch_size=25, validation_split=0.2)

Epoch 1/200
230/230 [==============================] - 5s 6ms/step - loss: 1.0405 - mae: 0.7982 - mse: 1.0405 - val_loss: 0.6077 - val_mae: 0.6232 - val_mse: 0.6077
Epoch 2/200
230/230 [==============================] - 1s 5ms/step - loss: 0.6422 - mae: 0.6285 - mse: 0.6422 - val_loss: 0.4724 - val_mae: 0.5525 - val_mse: 0.4724
Epoch 3/200
230/230 [==============================] - 1s 5ms/step - loss: 0.5794 - mae: 0.5967 - mse: 0.5794 - val_loss: 0.4691 - val_mae: 0.5485 - val_mse: 0.4691
Epoch 4/200
230/230 [==============================] - 1s 5ms/step - loss: 0.5657 - mae: 0.5853 - mse: 0.5657 - val_loss: 0.4719 - val_mae: 0.5493 - val_mse: 0.4719
Epoch 5/200
230/230 [==============================] - 1s 5ms/step - loss: 0.5601 - mae: 0.5864 - mse: 0.5601 - val_loss: 0.4274 - val_mae: 0.5184 - val_mse: 0.4274
Epoch 6/200
230/230 [==============================] - 1s 5ms/step - loss: 0.6266 - mae: 0.6119 - mse: 0.6266 - val_loss: 0.4556 - val_mae: 0.5368 - val_mse: 0.4556
Epoch 7/20

In [37]:
history2 = model_wording.fit(features, target2, epochs=200,batch_size=25, validation_split=0.2)

Epoch 1/200
230/230 [==============================] - 3s 5ms/step - loss: 1.0603 - mae: 0.8132 - mse: 1.0603 - val_loss: 0.6877 - val_mae: 0.6600 - val_mse: 0.6877
Epoch 2/200
230/230 [==============================] - 1s 5ms/step - loss: 0.8268 - mae: 0.7182 - mse: 0.8268 - val_loss: 0.6884 - val_mae: 0.6686 - val_mse: 0.6884
Epoch 3/200
230/230 [==============================] - 1s 5ms/step - loss: 0.8133 - mae: 0.7134 - mse: 0.8133 - val_loss: 0.6032 - val_mae: 0.6270 - val_mse: 0.6032
Epoch 4/200
230/230 [==============================] - 1s 5ms/step - loss: 0.7988 - mae: 0.7118 - mse: 0.7988 - val_loss: 0.6595 - val_mae: 0.6614 - val_mse: 0.6595
Epoch 5/200
230/230 [==============================] - 1s 5ms/step - loss: 0.7589 - mae: 0.6901 - mse: 0.7589 - val_loss: 0.6349 - val_mae: 0.6457 - val_mse: 0.6349
Epoch 6/200
230/230 [==============================] - 1s 5ms/step - loss: 0.7612 - mae: 0.6942 - mse: 0.7612 - val_loss: 0.6496 - val_mae: 0.6450 - val_mse: 0.6496
Epoch 7/20

In [38]:
evaluate_on_train_content = model_content.evaluate(features, target1)
evaluate_on_train_wording = model_wording.evaluate(features, target2)

224/224 [==============================] - 1s 3ms/step - loss: 0.5255 - mae: 0.5752 - mse: 0.5255


In [39]:
print('evaluate_on_train_content', evaluate_on_train_content)
print('evaluate_on_train_wording',evaluate_on_train_wording)

evaluate_on_train_content [0.706400990486145, 0.7058922052383423, 0.706400990486145]
evaluate_on_train_wording [0.525549590587616, 0.5751713514328003, 0.525549590587616]


In [40]:
content_prediction = model_content.predict(features)
wording_prediction = model_wording.predict(features)

224/224 [==============================] - 0s 2ms/step


### Predict on test

In [41]:
test_pred_content = model_content.predict(features_for_test)
test_pred_wording = model_wording.predict(features_for_test)

1/1 [==============================] - 0s 20ms/step


## submission

In [42]:
test_pred_content = test_pred_content.reshape(-1)
test_pred_wording = test_pred_wording.reshape(-1)

In [43]:
submission = pd.DataFrame({
    'student_id' : test['student_id'],
    'content' : test_pred_content,
    'wording' : test_pred_wording
})

In [44]:
submission.to_csv('submission.csv', index=False)

In [45]:
submission.head()

,student_id,content,wording
0,000000ffffff,-0.764213,-1.043358
1,222222cccccc,-0.764213,-1.043358
2,111111eeeeee,-0.764213,-1.043358
3,333333dddddd,-0.764213,-1.043358
